# Speed dating experiment

Predicting decision by partner by using the dec_o variable and using the <b>k-nearest neighbor algorithm.</b>

In [100]:
import seaborn as sns
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

df = pd.read_csv('SpeedDatingData.csv', encoding ='unicode_escape') # unicode because the file didnt work
df.head(5)

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


# Selecting the variable

In [101]:
df_subset = df[["dec_o","age", "samerace","fun_o","match","attr_o","like_o","prob_o"]]
df_subset = df_subset.dropna()
df_subset

,dec_o,age,samerace,fun_o,match,attr_o,like_o,prob_o
0,0,21.0,0,8.0,0,6.0,7.0,4.0
1,0,21.0,0,7.0,0,7.0,8.0,4.0
2,1,21.0,1,10.0,1,10.0,10.0,10.0
3,1,21.0,0,8.0,1,7.0,7.0,7.0
4,1,21.0,0,6.0,1,8.0,8.0,6.0
...,...,...,...,...,...,...,...,...
8373,1,25.0,0,2.0,0,10.0,6.0,1.0
8374,0,25.0,0,3.0,0,6.0,2.0,2.0
8375,0,25.0,0,2.0,0,2.0,2.0,1.0
8376,1,25.0,0,5.0,0,5.0,6.0,4.0


In [102]:
df_subset.corr()

,dec_o,age,samerace,fun_o,match,attr_o,like_o,prob_o
dec_o,1.000000,-0.050464,0.022131,0.413847,0.522382,0.491419,0.513857,0.311893
age,-0.050464,1.000000,-0.004239,-0.041268,-0.034826,-0.053575,-0.043676,-0.009172
samerace,0.022131,-0.004239,1.000000,0.012596,0.011421,0.052488,0.043085,0.088332
fun_o,0.413847,-0.041268,0.012596,1.000000,0.276611,0.586954,0.689014,0.381084
match,0.522382,-0.034826,0.011421,0.276611,1.000000,0.265609,0.307651,0.256246
attr_o,0.491419,-0.053575,0.052488,0.586954,0.265609,1.000000,0.669170,0.270273
like_o,0.513857,-0.043676,0.043085,0.689014,0.307651,0.669170,1.000000,0.470465
prob_o,0.311893,-0.009172,0.088332,0.381084,0.256246,0.270273,0.470465,1.000000


These variables seem the most correlated to me. 

# k-nearest neighbor algorithm

The k-nearest neighbor algorithm is very simple. It recognized the pattern in the data and clusters (hence the K). It assigns the data to the class of that single nearest neighbor.

# Split the data set into a training and test set

In [103]:
#from sklearn.preprocessing import normalize #get the function needed to normalize our data.
#df2 = df_dummies[['Age', 'SibSp','male','Pclass']]

X = df_subset[["age", "samerace","fun_o","match","attr_o","like_o","prob_o"]] #create the X matrix
#X = normalize(X) #normalize the matrix to put everything on the same scale
y = df_subset['dec_o'] #create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #split the data, store it into different variables

In [104]:
from sklearn.neighbors import KNeighborsClassifier #the object class we need

knn = KNeighborsClassifier(n_neighbors=6) #create a KNN-classifier with 5 neighbors (default)
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data
knn.score(X_test, y_test) #calculate the fit on the test data
# the score is the accurracy

0.7696245733788396

The <b>accurracy</b> of the model is 79%

In [105]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[1206,  162],
       [ 378,  598]], dtype=int64)

In [106]:
#In order to read it easily , let's make a dataframe out of it, and add labels to it.
conf_matrix = pd.DataFrame(cm, index=['actual yes to next date', 'actual no to next date'], columns = ['predicted yes', 'predicted no']) 
conf_matrix

,predicted yes,predicted no
actual yes to next date,1206,162
actual no to next date,378,598


$recall = \frac{1206}{1206+162} = .88$

88% the predicted 'decisions yes' are actually 'no' decisions. 


$precision = \frac{1206}{1206+378} = .76$

76% of the decisions 'yes' are predicted as decisions 'yes'



